
# Final Project
# Big Data - Fall 2021

## By ma5416, sp6322 & lk2849

### NYPD Complaint Data Historic Public Safety
### This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD)
### https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.2.4
matplotlib version: 3.3.4
NumPy version: 1.20.1
SciPy version: 1.6.2
IPython version: 7.22.0
scikit-learn version: 0.24.1
-------------------------


In [2]:
#url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_src = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD_Complaint_Data_Historic.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

File NYPD_Complaint_Data_Historic.csv has already been downloaded


In [4]:
pip install openclean

  Using cached openclean-0.2.1-py3-none-any.whl (5.2 kB)
  Using cached openclean_core-0.4.1-py3-none-any.whl (267 kB)
  Using cached refdata-0.2.0-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from openclean-core==0.4.1->openclean) (0.18.2)
  Using cached histore-0.4.1-py3-none-any.whl (109 kB)

  Using cached jellyfish-0.8.9-cp38-cp38-win_amd64.whl (28 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached flowserv_core-0.9.2-py3-none-any.whl (260 kB)
  Using cached passlib-1.7.4-py2.py3-none-any.whl (525 kB)
  Using cached pyyaml_include-1.2.post2-py3-none-any.whl (20 kB)
  Using cached GitPython-3.1.24-py3-none-any.whl (180 kB)
  Using cached pooch-1.5.2-py3-none-any.whl (57 kB)
  Using cached tableprint-0.9.1-py3-none-any.whl (6.8 kB)
  Using cached datasize-1.0.0-py2.py3-none-any.whl
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)

In [5]:
from openclean.pipeline import stream
ds_full = stream(fn_dst)

In [6]:
print(f'{ds_full.count():,} rows.')

7,375,993 rows.


In [7]:
ds_full.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,394506329,12/31/2019,17:30:00,,,32,12/31/2019,118,DANGEROUS WEAPONS,793,...,,,40.82092679700002,-73.94332421899996,"(40.82092679700002, -73.94332421899996)",PATROL BORO MAN NORTH,,UNKNOWN,UNKNOWN,E
1,968873685,12/29/2019,16:31:00,12/29/2019,16:54:00,47,12/29/2019,113,FORGERY,729,...,,,40.885701406000074,-73.86164032499995,"(40.885701406000074, -73.86164032499995)",PATROL BORO BRONX,,UNKNOWN,UNKNOWN,E
2,509837549,12/15/2019,18:45:00,,,109,12/29/2019,578,HARRASSMENT 2,638,...,M,,40.74228115600005,-73.81982408,"(40.74228115600005, -73.81982408)",PATROL BORO QUEENS NORTH,,25-44,WHITE HISPANIC,F
3,352454313,12/28/2019,01:00:00,,,47,12/28/2019,126,MISCELLANEOUS PENAL LAW,117,...,M,,40.87531145100007,-73.84754521099995,"(40.87531145100007, -73.84754521099995)",PATROL BORO BRONX,,UNKNOWN,UNKNOWN,E
4,248803469,09/05/2008,21:41:00,,,,09/05/2008,101,MURDER & NON-NEGL. MANSLAUGHTER,,...,,,40.698827283,-73.938819047,"(40.698827283, -73.938819047)",,,25-44,BLACK,M
5,293718737,12/27/2019,22:00:00,,,9,12/27/2019,107,BURGLARY,223,...,,,40.72075882100006,-73.98046642299995,"(40.72075882100006, -73.98046642299995)",PATROL BORO MAN SOUTH,,UNKNOWN,UNKNOWN,M
6,552685226,12/27/2019,20:10:00,12/27/2019,20:15:00,79,12/27/2019,117,DANGEROUS DRUGS,521,...,,,40.695958362000056,-73.93763162199998,"(40.695958362000056, -73.93763162199998)",PATROL BORO BKLYN NORTH,,UNKNOWN,UNKNOWN,E
7,134037758,12/26/2019,20:00:00,12/27/2019,07:15:00,101,12/27/2019,341,PETIT LARCENY,321,...,,,40.61157006600007,-73.74736517199995,"(40.61157006600007, -73.74736517199995)",PATROL BORO QUEENS SOUTH,,25-44,WHITE HISPANIC,F
8,855385879,12/26/2019,19:57:00,,,44,12/26/2019,361,OFF. AGNST PUB ORD SENSBLTY &,639,...,F,,40.84023413800003,-73.91768411399994,"(40.84023413800003, -73.91768411399994)",PATROL BORO BRONX,,18-24,BLACK,F
9,241602326,12/25/2019,23:00:00,12/26/2019,14:25:00,20,12/26/2019,341,PETIT LARCENY,321,...,,,40.77287456000005,-73.98954212299998,"(40.77287456000005, -73.98954212299998)",PATROL BORO MAN NORTH,,45-64,BLACK,M


In [8]:
from openclean.profiling.column import DefaultColumnProfiler

#profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

In [9]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
CMPLNT_NUM,7375993,0,7373143,9.996136e-01,22.813633
CMPLNT_FR_DT,7375993,655,8606,1.166862e-03,12.425578
CMPLNT_FR_TM,7375993,48,1441,1.953648e-04,8.136466
CMPLNT_TO_DT,7375993,1704204,6825,1.203324e-03,12.417984
CMPLNT_TO_TM,7375993,1699541,1441,2.538558e-04,8.862856
ADDR_PCT_CD,7375993,2166,77,1.044234e-05,6.148690
RPT_DT,7375993,0,5479,7.428152e-04,12.405384
KY_CD,7375993,0,74,1.003255e-05,4.170727
OFNS_DESC,7375993,18823,71,9.650450e-06,4.006583
PD_CD,7375993,6278,432,5.861828e-05,5.913459


In [11]:
# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds_full.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'CMPLNT_NUM' (int)
  'CMPLNT_FR_DT' (date)
  'CMPLNT_FR_TM' (date)
  'CMPLNT_TO_DT' (date)
  'CMPLNT_TO_TM' (date)
  'ADDR_PCT_CD' (int)
  'RPT_DT' (date)
  'KY_CD' (int)
  'OFNS_DESC' (str)
  'PD_CD' (int)
  'PD_DESC' (str)
  'CRM_ATPT_CPTD_CD' (str)
  'LAW_CAT_CD' (str)
  'BORO_NM' (str)
  'LOC_OF_OCCUR_DESC' (str)
  'PREM_TYP_DESC' (str)
  'JURIS_DESC' (str)
  'JURISDICTION_CODE' (int)
  'PARKS_NM' (str)
  'HADEVELOPT' (str)
  'HOUSING_PSA' (int)
  'X_COORD_CD' (int)
  'Y_COORD_CD' (int)
  'SUSP_AGE_GROUP' (int)
  'SUSP_RACE' (str)
  'SUSP_SEX' (str)
  'TRANSIT_DISTRICT' (int)
  'Latitude' (float)
  'Longitude' (float)
  'Lat_Lon' (str)
  'PATROL_BORO' (str)
  'STATION_NAME' (str)
  'VIC_AGE_GROUP' (int)
  'VIC_RACE' (str)
  'VIC_SEX' (str)


In [12]:
profiles.minmax('CMPLNT_FR_DT')

,min,max
date,1010-05-14 00:00:00,2020-12-31


In [13]:
profiles.minmax('CMPLNT_FR_TM')

,min,max
date,2021-11-15,2021-11-15 23:59:00


In [14]:
profiles.minmax('CMPLNT_TO_DT')

,min,max
date,1010-10-15 00:00:00,2090-04-06


In [15]:
profiles.minmax('CMPLNT_TO_TM')

,min,max
date,2021-11-15 00:00:00,2021-11-15 23:59:00
str,24:00:00,24:00:00


In [16]:
profiles.minmax('ADDR_PCT_CD')

,min,max
int,1,123


In [18]:
# Print the most frequent values in column 'DDR_PCT_CD'

profiles.column('ADDR_PCT_CD').get('topValues')

[('75', 237049),
 ('43', 189629),
 ('44', 184214),
 ('40', 178413),
 ('14', 166925),
 ('46', 155839),
 ('52', 154649),
 ('73', 151060),
 ('120', 145011),
 ('67', 139888)]

In [19]:
profiles.minmax('RPT_DT')

,min,max
date,2006-01-01,2020-12-31


In [20]:
profiles.minmax('KY_CD')

,min,max
int,101,881


In [21]:
# Print the most frequent values in column 'KY_CD'

profiles.column('KY_CD').get('topValues')

[('341', 1244164),
 ('578', 945459),
 ('344', 774250),
 ('109', 638981),
 ('351', 619342),
 ('361', 383375),
 ('235', 342576),
 ('106', 286139),
 ('105', 266449),
 ('107', 254293)]

In [22]:
# Print the most frequent values in column 'OFNS_DESC'

profiles.column('OFNS_DESC').get('topValues')

[('PETIT LARCENY', 1244155),
 ('HARRASSMENT 2', 945389),
 ('ASSAULT 3 & RELATED OFFENSES', 774177),
 ('CRIMINAL MISCHIEF & RELATED OF', 744951),
 ('GRAND LARCENY', 638972),
 ('DANGEROUS DRUGS', 427058),
 ('OFF. AGNST PUB ORD SENSBLTY &', 383332),
 ('FELONY ASSAULT', 286085),
 ('ROBBERY', 266447),
 ('BURGLARY', 254292)]

In [23]:
profiles.minmax('PD_CD')

,min,max
int,100,975


In [25]:
# Print the most frequent values in column ''

profiles.column('PD_CD').get('topValues')

[('101', 640218),
 ('638', 613331),
 ('639', 374673),
 ('333', 364252),
 ('637', 332066),
 ('338', 302018),
 ('254', 259062),
 ('321', 237811),
 ('109', 232452),
 ('259', 208063)]

In [26]:
# Print the most frequent values in column ''

profiles.column('PD_DESC').get('topValues')

[('ASSAULT 3', 640218),
 ('HARASSMENT,SUBD 3,4,5', 613331),
 ('AGGRAVATED HARASSMENT 2', 374673),
 ('LARCENY,PETIT FROM STORE-SHOPL', 364252),
 ('HARASSMENT,SUBD 1,CIVILIAN', 332066),
 ('LARCENY,PETIT FROM BUILDING,UN', 302018),
 ('MISCHIEF, CRIMINAL 4, OF MOTOR', 259062),
 ('LARCENY,PETIT FROM AUTO', 237811),
 ('ASSAULT 2,1,UNCLASSIFIED', 232452),
 ('CRIMINAL MISCHIEF,UNCLASSIFIED 4', 208063)]

In [ ]:
profiles.minmax('')

In [ ]:
# Print the most frequent values in column ''

profiles.column('').get('topValues')